In [1]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,Lambda,add,concatenate, BatchNormalization,GlobalAveragePooling2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.regularizers import l2
%matplotlib inline

import keras.backend as K
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
def grouped_convolution_block(input1, grouped_channels, cardinality, strides):
    init = input1
    group_list = []
    for c in range(cardinality):
        x = Lambda(lambda z: z[:, :, :, c * grouped_channels:(c + 1) * grouped_channels])(input1)
        x = Conv2D(grouped_channels, (3, 3), padding='same', use_bias=False, strides=(strides, strides), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(x)
        group_list.append(x)
    group_merge = concatenate(group_list, axis=-1)
    x = BatchNormalization(axis=-1)(group_merge)
    x = Activation('relu')(x)
    return x

def bottleneck_block(input1, filters, cardinality=32, strides=1):
    init = input1

    grouped_channels = int(filters / cardinality)
    if init._keras_shape[-1] != 2 * filters:
        init = Conv2D(filters * 2, (1, 1), padding='same', strides=(strides, strides),use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(init)
        init = BatchNormalization(axis=-1)(init)

    x = Conv2D(filters, (1, 1), padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(input1)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)

    x = grouped_convolution_block(x, grouped_channels, cardinality, strides)

    x = Conv2D(filters * 2, (1, 1), padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(x)
    x = BatchNormalization(axis=-1)(x)

    x = add([init, x])
    x = Activation('relu')(x)

    return x

#ResNeXt 50: depth = [3,4,6,3]
#ResNeXt 101: depth =[3,4,23,3]
#ResNeXt 152: depth =[3,8,36,3]
def ResNeXt(input_shape = (224, 224,3), depth=[3,4,23,3], cardinality=32, width=4):
    X_input = Input(input_shape)
    X = Conv2D(64, (7, 7), padding='same', use_bias=False, strides=(2, 2), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(X_input)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(X)
    
    filters = cardinality * width
    filters_list = []
    for i in range(len(depth)):
        filters_list.append(filters)
        filters *= 2 

    for i in range(depth[0]):
        X = bottleneck_block(X, filters_list[0], cardinality, strides=1)

    depth = depth[1:]  
    filters_list = filters_list[1:]  

    for block_idx, n_i in enumerate(depth):
        for i in range(n_i):
            if i == 0:
                X = bottleneck_block(X, filters_list[block_idx], cardinality, strides=2)
            else:
                X = bottleneck_block(X, filters_list[block_idx], cardinality, strides=1)
    X = GlobalAveragePooling2D()(X)
    X = Dense(2, use_bias=False, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='ResNeXt')

    return model

In [3]:
model = ResNeXt()

In [37]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_4625 (Conv2D)            (None, 112, 112, 64) 9408        input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_342 (BatchN (None, 112, 112, 64) 256         conv2d_4625[0][0]                
__________________________________________________________________________________________________
activation_320 (Activation)     (None, 112, 112, 64) 0           batch_normalization_342[0][0]    
__________________________________________________________________________________________________
max_poolin

                                                                 conv2d_4908[0][0]                
                                                                 conv2d_4909[0][0]                
                                                                 conv2d_4910[0][0]                
                                                                 conv2d_4911[0][0]                
                                                                 conv2d_4912[0][0]                
                                                                 conv2d_4913[0][0]                
                                                                 conv2d_4914[0][0]                
                                                                 conv2d_4915[0][0]                
                                                                 conv2d_4916[0][0]                
                                                                 conv2d_4917[0][0]                
          

__________________________________________________________________________________________________
lambda_4909 (Lambda)            (None, 14, 14, 16)   0           activation_369[0][0]             
__________________________________________________________________________________________________
lambda_4910 (Lambda)            (None, 14, 14, 16)   0           activation_369[0][0]             
__________________________________________________________________________________________________
lambda_4911 (Lambda)            (None, 14, 14, 16)   0           activation_369[0][0]             
__________________________________________________________________________________________________
lambda_4912 (Lambda)            (None, 14, 14, 16)   0           activation_369[0][0]             
__________________________________________________________________________________________________
lambda_4913 (Lambda)            (None, 14, 14, 16)   0           activation_369[0][0]             
__________

__________________________________________________________________________________________________
conv2d_5403 (Conv2D)            (None, 14, 14, 16)   2304        lambda_5114[0][0]                
__________________________________________________________________________________________________
conv2d_5404 (Conv2D)            (None, 14, 14, 16)   2304        lambda_5115[0][0]                
__________________________________________________________________________________________________
conv2d_5405 (Conv2D)            (None, 14, 14, 16)   2304        lambda_5116[0][0]                
__________________________________________________________________________________________________
conv2d_5406 (Conv2D)            (None, 14, 14, 16)   2304        lambda_5117[0][0]                
__________________________________________________________________________________________________
conv2d_5407 (Conv2D)            (None, 14, 14, 16)   2304        lambda_5118[0][0]                
__________

add_133 (Add)                   (None, 7, 7, 2048)   0           batch_normalization_436[0][0]    
                                                                 batch_normalization_439[0][0]    
__________________________________________________________________________________________________
activation_413 (Activation)     (None, 7, 7, 2048)   0           add_133[0][0]                    
__________________________________________________________________________________________________
conv2d_5684 (Conv2D)            (None, 7, 7, 1024)   2097152     activation_413[0][0]             
__________________________________________________________________________________________________
batch_normalization_440 (BatchN (None, 7, 7, 1024)   4096        conv2d_5684[0][0]                
__________________________________________________________________________________________________
activation_414 (Activation)     (None, 7, 7, 1024)   0           batch_normalization_440[0][0]    
__________

In [38]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,epochs = 900, batch_size=32,validation_data=(x_test,y_test), shuffle = True)